In [43]:
import torch

In [16]:
ratings_path = 'data/ml-1m/ratings.dat'


In [59]:
import pandas as pd
ratings = pd.read_csv(ratings_path, names=['user', 'item', 'rating', 'timestamp'], sep='::', engine='python')
ratings

,user,item,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [30]:
uidx_map = {user:i for i, user in enumerate(set(ratings['user']))}
iidx_map = {item:i for i, item in enumerate(set(ratings['item']))}
ratings['uidx'] = ratings['user'].map(uidx_map)
ratings['iidx'] = ratings['item'].map(iidx_map)

In [31]:
n_user = ratings['uidx'].nunique()
n_item = ratings['iidx'].nunique()

In [33]:
import torch.nn as nn

In [ ]:
emb_dim = 64


In [56]:
class NeuMF(nn.Module):
    def __init__(self, n_user, n_item, emb_dim):
        super().__init__()
        # GMF
        self.GMF_user = nn.Embedding(n_user, emb_dim)
        self.GMF_item = nn.Embedding(n_item, emb_dim)

        # MLP
        self.MLP_user = nn.Embedding(n_user, emb_dim)
        self.MLP_item = nn.Embedding(n_item, emb_dim)
        self.MLP_linear = nn.Sequential(
            nn.Linear(emb_dim*2, emb_dim//2),
            nn.ReLU(),
            nn.Linear(emb_dim//2, emb_dim//(2*2)),
            nn.ReLU(),
            nn.Linear(emb_dim//(2*2), emb_dim//(2*2*2)),
        )

        # output layer
        self.output_layer = nn.Linear(emb_dim + (emb_dim//(2*2*2)), 1)

    def forward(self, user, item):
        # GMF
        phi_gmf = torch.mul(self.GMF_user(user), self.GMF_item(item))

        # MLP
        concat = torch.cat([self.MLP_user(user), self.MLP_item(item)], dim=1)
        phi_mlp = self.MLP_linear(concat)
        
        # output layer
        output = self.output_layer(torch.cat([phi_gmf, phi_mlp], dim=1))
        logit = torch.sigmoid(output)

        return logit




In [57]:
model = NeuMF(n_user, n_item, emb_dim)

In [58]:
model(torch.tensor([3]), torch.tensor([4]))

tensor([[0.4236]], grad_fn=<SigmoidBackward0>)